<a href="https://colab.research.google.com/github/zeon62/TinyML/blob/main/struct1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TENSORFLOW VERSION**

In [ ]:
!pip install tensorflow==2.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
SAMPLES = 1000
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

x_values = np.random.uniform(low=0, high=2*math.pi, size=SAMPLES)
np.random.shuffle(x_values)
y_values = np.sin(x_values)

In [ ]:
y_values += 0.1 * np.random.randn(*y_values.shape)

In [ ]:
TRAIN_SPLIT = int(0.6*SAMPLES)
TEST_SPLIT = int(0.2*SAMPLES + TRAIN_SPLIT)

x_train, x_validate, x_test = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_validate, y_test = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

In [ ]:
from tensorflow.keras import layers
model_1 = tf.keras.Sequential()
model_1.add(layers.Dense(16, activation = 'relu', input_shape = (1,)))
model_1.add(layers.Dense(1))
model_1.compile(optimizer = 'rmsprop', loss='mse', metrics=['mae'])
model_1.save("model.h5")
model_1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                32        
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 49
Trainable params: 49
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history_1 = model_1.fit(x_train, y_train, epochs=100, batch_size=16, validation_data=(x_validate, y_validate))

Epoch 1/100
38/38 [==============================] - 1s 9ms/step - loss: 0.4740 - mae: 0.6116 - val_loss: 0.4833 - val_mae: 0.6116
Epoch 2/100
38/38 [==============================] - 0s 5ms/step - loss: 0.4448 - mae: 0.5910 - val_loss: 0.4578 - val_mae: 0.5943
Epoch 3/100
38/38 [==============================] - 0s 4ms/step - loss: 0.4220 - mae: 0.5736 - val_loss: 0.4352 - val_mae: 0.5778
Epoch 4/100
38/38 [==============================] - 0s 4ms/step - loss: 0.4009 - mae: 0.5578 - val_loss: 0.4138 - val_mae: 0.5614
Epoch 5/100
38/38 [==============================] - 0s 5ms/step - loss: 0.3807 - mae: 0.5421 - val_loss: 0.3917 - val_mae: 0.5442
Epoch 6/100
38/38 [==============================] - 0s 6ms/step - loss: 0.3597 - mae: 0.5262 - val_loss: 0.3708 - val_mae: 0.5283
Epoch 7/100
38/38 [==============================] - 0s 4ms/step - loss: 0.3409 - mae: 0.5112 - val_loss: 0.3514 - val_mae: 0.5131
Epoch 8/100
38/38 [==============================] - 0s 4ms/step - loss: 0.3230 - m

**CONVERT**

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_1)
tflite_model = converter.convert()

open("sine_model.tflite", "wb").write(tflite_model)

1804

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the FULLY_CONNECTED op takes
tensor #0 and tensor #3 and tensor #2 as input and produces tensor #5 as output.

Subgraph#0 main(T#0) -> [T#7]
  Op#0 FULLY_CONNECTED(T#0, T#3, T#2) -> [T#5]
  Op#1 SOFTMAX(T#5) -> [T#6]
  Op#2 FULLY_CONNECTED(T#6, T#4, T#1) -> [T#7]

Tensors of Subgraph#0
  T#0(serving_default_dense_6_input:0) shape_signature:[-1, 1], type:FLOAT32
  T#1(sequential_3/dense_7/BiasAdd/ReadVariableOp) shape:[1], type:FLOAT32 RO 4 bytes
  T#2(sequential_3/dense_6/BiasAdd/ReadVariableOp) shape:[16], type:FLOAT32 RO 64 bytes
  T#3(sequential_3/dense_6/MatMul) shape:[16, 1], type:FLOAT32 RO 64 bytes
  T#4(sequential_3/dense_7/MatMul) shape:[1, 16], type:FLOAT32 RO 64 bytes
  T#5(sequential_3/dense_6/MatMul;sequential_3/dense_6/BiasAdd) shape_signature:[-1, 16], type:FLOAT32
  T#6(sequential_3/dense_6/Softmax) shape_s

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_1)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_generator():
    for value in x_test:
      yield[np.array(value, dtype = np.float32, ndmin = 2)]
converter.representative_dataset = representative_dataset_generator
tflite_model = converter.convert()

open("sine_model_quantized.tflite", "wb").write(tflite_model)

/usr/local/lib/python3.8/dist-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


2328

In [ ]:
import os
basic_model_size = os.path.getsize("sine_model.tflite")
print("Basic model is %d bytes" % basic_model_size)
quantized_model_size = os.path.getsize("sine_model_quantized.tflite")
print("Quantized model is %d bytes" % quantized_model_size)
difference = basic_model_size - quantized_model_size
print("Difference is %d bytes" % difference)

Basic model is 1804 bytes
Quantized model is 2328 bytes
Difference is -524 bytes


In [ ]:
!apt-get -qq install xxd
!xxd -i sine_model.tflite > sine_model.cc
!xxd -i sine_model_quantized.tflite > sine_model_quantized.cc

In [ ]:
!xxd -c 16 sine_model_quantized.tflite > testq.cc
!xxd -c 16 sine_model.tflite > test.cc

In [ ]:
from google.colab import files
files.download('model.h5')
files.download('sine_model.cc')
files.download('sine_model.tflite')
files.download('sine_model_quantized.cc')
files.download('sine_model_quantized.tflite')
files.download('test.cc')
files.download('testq.cc')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)


=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #5 as output.

Subgraph#0 main(T#0) -> [T#9]
  Op#0 QUANTIZE(T#0) -> [T#5]
  Op#1 FULLY_CONNECTED(T#5, T#1, T#2) -> [T#6]
  Op#2 SOFTMAX(T#6) -> [T#7]
  Op#3 FULLY_CONNECTED(T#7, T#3, T#4) -> [T#8]
  Op#4 DEQUANTIZE(T#8) -> [T#9]

Tensors of Subgraph#0
  T#0(serving_default_dense_6_input:0) shape_signature:[-1, 1], type:FLOAT32
  T#1(sequential_3/dense_6/MatMul) shape:[16, 1], type:INT8 RO 16 bytes
  T#2(sequential_3/dense_6/BiasAdd/ReadVariableOp) shape:[16], type:INT32 RO 64 bytes
  T#3(sequential_3/dense_7/MatMul) shape:[1, 16], type:INT8 RO 16 bytes
  T#4(sequential_3/dense_7/BiasAdd/ReadVariableOp) shape:[1], type:INT32 RO 4 bytes
  T#5(tfl.quantize) shape_signature:[-1, 1], type:INT8
  T#6(sequential_3/dense_6/MatMul;sequential_3/dense_6/BiasAdd) shape_sign